In [1]:
import logging
import time
import re
from pathlib import Path

import requests
from aequilibrae.parameters import Parameters
from aequilibrae.context import get_logger
import gc
import importlib.util as iutil
from aequilibrae.utils import WorkerThread

import duckdb
import geopandas as gpd
import subprocess
import os
from typing import Union

In [2]:
def initialise_duckdb_spatial():
    conn = duckdb.connect()
    c = conn.cursor()

    c.execute(
        """INSTALL spatial; 
            INSTALL httpfs;
            INSTALL parquet;
        """
    )
    c.execute(
        """LOAD spatial;
        LOAD parquet;
        SET s3_region='us-west-2';
        """
    )
    return c


In [19]:
pth = r'E:\theme=transportation\type=segment'
pth2 = r'C:\Users\penny\git\Aequilibrae\theme=transportation\type=segment\transportation_parquet.zstd.parquet'
airlie_bbox = [148.7077, -20.2780, 148.7324, -20.2621 ]

sql = f"""
    COPY(
    SELECT *
    FROM read_parquet('{pth}/*', union_by_name=True)
    WHERE bbox.minx > '{airlie_bbox[0]}'
        AND bbox.maxx < '{airlie_bbox[2]}'
        AND bbox.miny > '{airlie_bbox[1]}'
        AND bbox.maxy < '{airlie_bbox[3]}')
    TO '{pth2}'
    (FORMAT 'parquet', COMPRESSION 'ZSTD')
"""
c = initialise_duckdb_spatial()
g = c.execute(sql)
g 
g.df()

,Count
0,281


In [10]:
pth = r'E:\theme=transportation\type=segment'
pth2 = r'C:\Users\penny\git\Aequilibrae\theme=transportation\type=segment\transportation.zstd.parquet'
airlie_bbox = [148.7077, -20.2780, 148.7324, -20.2621 ]

sql = f"""
    COPY(
    SELECT 
        ST_GeomFromWKB(geometry) AS geom,
        *
    FROM read_parquet('{pth}/*', union_by_name=True, hive_partitioning=1)
    WHERE bbox.minx > '{airlie_bbox[0]}'
        AND bbox.maxx < '{airlie_bbox[2]}'
        AND bbox.miny > '{airlie_bbox[1]}'
        AND bbox.maxy < '{airlie_bbox[3]}')
    TO '{pth2}'
    WITH (FORMAT 'parquet', CODEC 'ZSTD')
"""
c = initialise_duckdb_spatial()
g2 = c.execute(sql)
g2
g2.df()

In [9]:
g2.df()

In [6]:
pth = r'E:\theme=transportation\type=segment'
pth2 = r'C:\Users\penny\git\Aequilibrae\theme=transportation\type=segment\transportation3.parquet'
airlie_bbox = [148.7077, -20.2780, 148.7324, -20.2621 ]

sql = f"""
    COPY(
    SELECT 
        *
    FROM read_parquet('{pth}/*', union_by_name=True, hive_partitioning=1)
    WHERE bbox.minx > '{airlie_bbox[0]}'
        AND bbox.maxx < '{airlie_bbox[2]}'
        AND bbox.miny > '{airlie_bbox[1]}'
        AND bbox.maxy < '{airlie_bbox[3]}')
    TO '{pth2}'
    WITH (FORMAT 'parquet', CODEC 'ZSTD')
"""
c = initialise_duckdb_spatial()
g2 = c.execute(sql)
g2
g2.df()

RuntimeError: Query interrupted

In [ ]:
pth = r'E:\theme=transportation\type=segment\part-00232-8d133ca6-6cbd-48b8-87d5-a0850a2ba489.c003.zstd.parquet'
pth2 = r'C:\Users\penny\git\Aequilibrae\theme=transportation\type=segment\transportation_parquet.zstd.parquet'
airlie_bbox = [148.7077, -20.2780, 148.7324, -20.2621 ]

sql = f"""
    COPY(
    SELECT *
    FROM read_parquet('{pth}', union_by_name=True))
    TO '{pth2}'
    (FORMAT 'parquet', COMPRESSION 'ZSTD')
"""
initialise_duckdb_spatial().execute(sql).df()

,Count
0,88731


In [17]:
pth = r'E:\theme=transportation\type=segment\part-00232-8d133ca6-6cbd-48b8-87d5-a0850a2ba489.c003.zstd.parquet'
pth2 = r'C:\Users\penny\git\Aequilibrae\theme=transportation\type=segment\transportation_parquet.zstd.parquet'
airlie_bbox = [148.7077, -20.2780, 148.7324, -20.2621 ]

sql = f"""
   
    SELECT geometry
    FROM read_parquet('{pth}', union_by_name=True)
"""
initialise_duckdb_spatial().execute(sql).df()

,geometry
0,"[0, 0, 0, 0, 2, 0, 0, 0, 2, 64, 97, 27, 247, 1..."


In [15]:
from shapely.geometry import LineString
import shapely.wkb as wkb

# Assuming 'serialized_geometry' is the given serialized format
serialized_geometry = bytearray(b'\x00\x00\x00\x00\x02\x00\x00\x00\x14@a\x1c%t-\xcfF@B\x9d\xa0{5*\x84@a\x1c+\x9fU\x9b=@B\x9d\xaah\xf4\xb6 @a\x1c-\x01\xc0\xca`@B\x9d\xad[\xee=`@a\x1c.\x81\x88*\xdc@B\x9d\xb1\xc8d\x88@@a\x1c.\xf6\xf8\xf0A@B\x9d\xb3t62\xc2@a\x1c/@_k\xa0@B\x9d\xb5I\xf9HV@a\x1c/\x96[ \xb8@B\x9d\xba\xdc\t\x80\xb2@a\x1c/\xf0\x88\x93\xb8@B\x9d\xbdjY:.@a\x1c0\x92\x03\xa3#@B\x9d\xbf\xd7\x1b\x04h@a\x1c15\x97\x91\x82@B\x9d\xc1\x1eB\xe1&@a\x1c3j%7\xc8@B\x9d\xc3\x97\x99\xe5\x19@a\x1c4R\x82\x83\xd3@B\x9d\xc4\xd2,\x88\x1e@a\x1c5\x02\xab\xab\xeb@B\x9d\xc6u\x9a\xb6\xd0@a\x1c7AJM+@B\x9d\xce\xc8O\x8f\x8a@a\x1c9GIj\xad@B\x9d\xd5<\xdd\xd6\xe0@a\x1c:\xd4|\xc4w@B\x9d\xd9\x96\x08\xeba@a\x1c;+\x19\x89>@B\x9d\xda7\xefZ\x96@a\x1c?\xfb\x98\x923@B\x9d\xe0\x8a\xef\xb2\xab@a\x1cC")\x1f\xb4@B\x9d\xe4\xc5\x11\x16\xa9@a\x1cDoe\xe9i@B\x9d\xe6\xd3\xdf\x0f\xc5')

# Convert to bytes
serialized_geometry_bytes = bytes(serialized_geometry)

# Use shapely.wkb.loads to convert the serialized geometry to a LineString
line_string = wkb.loads(serialized_geometry_bytes)

# Print the LineString
print(line_string)



LINESTRING (136.879572 37.23146, 136.880325 37.231763, 136.880494 37.231853, 136.880677 37.231988, 136.880733 37.232039, 136.880768 37.232095, 136.880809 37.232265, 136.880852 37.232343, 136.880929 37.232417, 136.881007 37.232456, 136.8812762 37.2325315, 136.881387 37.232569, 136.881471 37.232619, 136.881745 37.232873, 136.881992 37.23307, 136.8821814 37.2332027, 136.8822227 37.233222, 136.8828104 37.233415, 136.883195 37.233544, 136.8833539 37.2336068)


In [23]:
pth = r'E:\theme=transportation\type=segment\part-00232-8d133ca6-6cbd-48b8-87d5-a0850a2ba489.c003.zstd.parquet'
pth2 = r'C:\Users\penny\git\Aequilibrae\theme=transportation\type=segment\transportation_parquet.zstd.parquet'
airlie_bbox = [148.7077, -20.2780, 148.7324, -20.2621 ]

sql = f"""
   
    SELECT geometry
    FROM read_parquet('{pth}', union_by_name=True)
"""
g = initialise_duckdb_spatial().execute(sql).df()

In [29]:
pth = r'E:\theme=transportation\type=segment\part-00232-8d133ca6-6cbd-48b8-87d5-a0850a2ba489.c003.zstd.parquet'
gdf = gpd.read_parquet(pth)
gdf

,id,geometry,bbox,subType,localityType,names,contextId,adminLevel,isoCountryCodeAlpha2,isoSubCountryCode,...,socials,emails,phones,brand,addresses,sourceTags,wikidata,surface,isSalt,isIntermittent
0,8a2e712b282ffff-17DFF24660C0674B,"LINESTRING (136.87391 37.21506, 136.87418 37.2...","{'minx': 136.8739104, 'maxx': 136.8741804, 'mi...",road,None,None,None,NaN,None,None,...,None,None,None,None,None,None,None,None,None,None
1,882e7174d3fffff-13FF4449D72183C2,"LINESTRING (136.87957 37.23146, 136.88032 37.2...","{'minx': 136.879572, 'maxx': 136.8833539, 'min...",road,None,None,None,NaN,None,None,...,None,None,None,None,None,None,None,None,None,None
2,892e7174d33ffff-17FE1106542131D2,"LINESTRING (136.88655 37.22834, 136.88672 37.2...","{'minx': 136.886546, 'maxx': 136.886733, 'miny...",road,None,None,None,NaN,None,None,...,None,None,None,None,None,None,None,None,None,None
3,872e71749ffffff-13CDE03A24646A31,"LINESTRING (136.86781 37.24933, 136.86757 37.2...","{'minx': 136.865495, 'maxx': 136.867811, 'miny...",road,None,None,None,NaN,None,None,...,None,None,None,None,None,None,None,None,None,None
4,872e7174dffffff-13FFCFC79C7F8D41,"LINESTRING (136.88335 37.23361, 136.88340 37.2...","{'minx': 136.8833539, 'maxx': 136.8888252, 'mi...",road,None,None,None,NaN,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88726,8804ebaf6dfffff-157B56F2364053C0,"LINESTRING (171.81161 69.73536, 171.81055 69.7...","{'minx': 171.8036842, 'maxx': 171.8116128, 'mi...",road,None,None,None,NaN,None,None,...,None,None,None,None,None,None,None,None,None,None
88727,8804ebaf61fffff-159FD531CC11F67B,"LINESTRING (171.83673 69.73090, 171.83653 69.7...","{'minx': 171.8275881, 'maxx': 171.836729, 'min...",road,None,None,None,NaN,None,None,...,None,None,None,None,None,None,None,None,None,None
88728,8704ebaf6ffffff-15BFEADF6048761D,"LINESTRING (171.82734 69.72528, 171.82774 69.7...","{'minx': 171.8273413, 'maxx': 171.8530772, 'mi...",road,None,None,None,NaN,None,None,...,None,None,None,None,None,None,None,None,None,None
88729,860db679fffffff-17EDF07478DAA881,"LINESTRING (173.78101 68.98210, 173.77863 68.9...","{'minx': 173.7595158, 'maxx': 173.7810087, 'mi...",road,None,None,None,NaN,None,None,...,None,None,None,None,None,None,None,None,None,None


In [60]:
gdf[: 1]["geometry"]

0    LINESTRING (-176.53736 -43.88630, -176.53686 -...
Name: geometry, dtype: geometry

In [45]:
test = gdf[1:5]
test

,id,geometry,bbox,subType,localityType,names,contextId,adminLevel,isoCountryCodeAlpha2,isoSubCountryCode,...,socials,emails,phones,brand,addresses,sourceTags,wikidata,surface,isSalt,isIntermittent
1,882e7174d3fffff-13FF4449D72183C2,"LINESTRING (136.87957 37.23146, 136.88032 37.2...","{'minx': 136.879572, 'maxx': 136.8833539, 'min...",road,None,None,None,NaN,None,None,...,None,None,None,None,None,None,None,None,None,None
2,892e7174d33ffff-17FE1106542131D2,"LINESTRING (136.88655 37.22834, 136.88672 37.2...","{'minx': 136.886546, 'maxx': 136.886733, 'miny...",road,None,None,None,NaN,None,None,...,None,None,None,None,None,None,None,None,None,None
3,872e71749ffffff-13CDE03A24646A31,"LINESTRING (136.86781 37.24933, 136.86757 37.2...","{'minx': 136.865495, 'maxx': 136.867811, 'miny...",road,None,None,None,NaN,None,None,...,None,None,None,None,None,None,None,None,None,None
4,872e7174dffffff-13FFCFC79C7F8D41,"LINESTRING (136.88335 37.23361, 136.88340 37.2...","{'minx': 136.8833539, 'maxx': 136.8888252, 'mi...",road,None,None,None,NaN,None,None,...,None,None,None,None,None,None,None,None,None,None


In [46]:
pth2 = r'C:\Users\penny\git\Aequilibrae\theme=transportation\type=segment\transportation_parquet.zstd.parquet'
test.to_parquet(pth,compression='zstd')

In [48]:
import geopandas as gpd
pth = r'E:\theme=transportation\type=segment\part-00232-8d133ca6-6cbd-48b8-87d5-a0850a2ba489.c003.zstd.parquet'
pth2 = r'C:\Users\penny\git\Aequilibrae\theme=transportation\type=segment\transportation_parquet.zstd.parquet'
airlie_bbox = [148.7077, -20.2780, 148.7324, -20.2621 ]

sql = f"""
   
    SELECT *
    FROM read_parquet('{pth}', union_by_name=True)
"""
initialise_duckdb_spatial().execute(sql)

In [1]:
import os
import geopandas as gpd

# Set the path to the 'parquet_data' directory
pth = r'E:\theme=transportation\type=segment'
pth2 = r'C:\Users\penny\git\Aequilibrae\theme=transportation\type=segment\transportation_data_files.zstd.parquet'

airlie_bbox = [148.7077, -20.2780, 148.7324, -20.2621]


# Loop through all the parquet files in the 'parquet_data' directory
for file in os.listdir(pth.replace("\\", "/")):
    if file.endswith('.parquet'):
        # Read the parquet file into a geopandas GeoDataFrame
        gdf = gpd.read_parquet(os.path.join(pth, file))

        # Filter the data using the WHERE command
        filtered_gdf = gdf[gdf['bbox'].apply(lambda bbox: airlie_bbox[0] <= bbox['minx'] <= airlie_bbox[2] and
                                                            airlie_bbox[0] <= bbox['maxx'] <= airlie_bbox[2] and
                                                            airlie_bbox[1] <= bbox['miny'] <= airlie_bbox[3] and
                                                            airlie_bbox[1] <= bbox['maxy'] <= airlie_bbox[3])]

        # Save the filtered data to a new parquet file
        filtered_gdf.to_parquet(pth2, compression='zstd')

In [14]:
# for i in 1:gdf.length:
gdf['bbox'][1]

{'minx': -176.236521,
 'maxx': -176.236365,
 'miny': -44.2445032,
 'maxy': -44.2442786}

In [63]:
pth = r'E:\theme=transportation\type=segment\part-00232-8d133ca6-6cbd-48b8-87d5-a0850a2ba489.c003.zstd.parquet'
gdf = gpd.read_parquet(pth)
gdf

,id,geometry,bbox,subType,localityType,names,contextId,adminLevel,isoCountryCodeAlpha2,isoSubCountryCode,...,socials,emails,phones,brand,addresses,sourceTags,wikidata,surface,isSalt,isIntermittent
0,8a2e712b282ffff-17DFF24660C0674B,"LINESTRING (136.87391 37.21506, 136.87418 37.2...","{'minx': 136.8739104, 'maxx': 136.8741804, 'mi...",road,None,None,None,NaN,None,None,...,None,None,None,None,None,None,None,None,None,None
1,882e7174d3fffff-13FF4449D72183C2,"LINESTRING (136.87957 37.23146, 136.88032 37.2...","{'minx': 136.879572, 'maxx': 136.8833539, 'min...",road,None,None,None,NaN,None,None,...,None,None,None,None,None,None,None,None,None,None
2,892e7174d33ffff-17FE1106542131D2,"LINESTRING (136.88655 37.22834, 136.88672 37.2...","{'minx': 136.886546, 'maxx': 136.886733, 'miny...",road,None,None,None,NaN,None,None,...,None,None,None,None,None,None,None,None,None,None
3,872e71749ffffff-13CDE03A24646A31,"LINESTRING (136.86781 37.24933, 136.86757 37.2...","{'minx': 136.865495, 'maxx': 136.867811, 'miny...",road,None,None,None,NaN,None,None,...,None,None,None,None,None,None,None,None,None,None
4,872e7174dffffff-13FFCFC79C7F8D41,"LINESTRING (136.88335 37.23361, 136.88340 37.2...","{'minx': 136.8833539, 'maxx': 136.8888252, 'mi...",road,None,None,None,NaN,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88726,8804ebaf6dfffff-157B56F2364053C0,"LINESTRING (171.81161 69.73536, 171.81055 69.7...","{'minx': 171.8036842, 'maxx': 171.8116128, 'mi...",road,None,None,None,NaN,None,None,...,None,None,None,None,None,None,None,None,None,None
88727,8804ebaf61fffff-159FD531CC11F67B,"LINESTRING (171.83673 69.73090, 171.83653 69.7...","{'minx': 171.8275881, 'maxx': 171.836729, 'min...",road,None,None,None,NaN,None,None,...,None,None,None,None,None,None,None,None,None,None
88728,8704ebaf6ffffff-15BFEADF6048761D,"LINESTRING (171.82734 69.72528, 171.82774 69.7...","{'minx': 171.8273413, 'maxx': 171.8530772, 'mi...",road,None,None,None,NaN,None,None,...,None,None,None,None,None,None,None,None,None,None
88729,860db679fffffff-17EDF07478DAA881,"LINESTRING (173.78101 68.98210, 173.77863 68.9...","{'minx': 173.7595158, 'maxx': 173.7810087, 'mi...",road,None,None,None,NaN,None,None,...,None,None,None,None,None,None,None,None,None,None


In [65]:
# Define the bounding box for the query
airlie_bbox = [148.7077, -20.2780, 148.7324, -20.2621]

# Query for geometries whose bounding boxes intersect with the airlie_bbox
result = gdf[gdf['geometry'].apply(lambda geom: geom.intersects(airlie_bbox))]

print(result)

Empty GeoDataFrame
Columns: [id, geometry, bbox, subType, localityType, names, contextId, adminLevel, isoCountryCodeAlpha2, isoSubCountryCode, defaultLanguage, drivingSide, version, updateTime, sources, isMaritime, geopolDisplay, localityId, height, numFloors, class, level, connectors, road, categories, confidence, websites, socials, emails, phones, brand, addresses, sourceTags, wikidata, surface, isSalt, isIntermittent]
Index: []

[0 rows x 37 columns]


In [68]:
from shapely.geometry import box

# Define the bounding box for the query
airlie_bbox = box(148.7077, -20.2780, 148.7324, -20.2621)

# Convert the bounding box coordinates to a Shapely box geometry
query_bbox = box(*airlie_bbox.bounds)

# Check if the bounding box in each row intersects with the query_bbox
result = gdf[gdf['bbox'].apply(lambda bbox: box(bbox['minx'], bbox['miny'], bbox['maxx'], bbox['maxy']).intersects(query_bbox))]

# Print the resulting GeoDataFrame
print(result)

Empty GeoDataFrame
Columns: [id, geometry, bbox, subType, localityType, names, contextId, adminLevel, isoCountryCodeAlpha2, isoSubCountryCode, defaultLanguage, drivingSide, version, updateTime, sources, isMaritime, geopolDisplay, localityId, height, numFloors, class, level, connectors, road, categories, confidence, websites, socials, emails, phones, brand, addresses, sourceTags, wikidata, surface, isSalt, isIntermittent]
Index: []

[0 rows x 37 columns]
